In [ ]:
from typing import Tuple, List, Dict
import functools
import time
import pprint
import os

In [ ]:
import jax
import jax.numpy as jnp

# predictably random behavior
RANDOM_KEY = jax.random.PRNGKey(42)

# what is this being run on?
print(f"jax.device_count() {jax.device_count()}")
print(f"jax.local_device_count() {jax.local_device_count()}")
for i, device in enumerate(jax.devices()):
    print(f" --- Found device: {i} ")
    print(f"\t device_kind: {device.device_kind}")
    print(f"\t platform: {device.platform}")
    print(f"\t host_id: {device.host_id}")


In [ ]:
def random_layer_params(
    input_size: int, output_size: int, random_key: jnp.DeviceArray, scale: float = 1e-2
) -> Tuple[jnp.DeviceArray]:
    """Creates a single layer of an MLP."""
    w_key, b_key = jax.random.split(random_key)
    return scale * jax.random.normal(
        w_key, (output_size, input_size)
    ), scale * jax.random.normal(b_key, (output_size,))


def init_network_params(
    layers: str, random_key: jnp.DeviceArray
) -> List[Tuple[jnp.DeviceArray]]:
    """Initialize a N-layer MLP."""
    layer_sizes: List[int] = [int(x) for x in layers.split(',')]
    layer_keys = jax.random.split(random_key, len(layer_sizes))
    network_params: List[Tuple[jnp.DeviceArray]] = []
    for in_size, out_size, key in zip(layer_sizes[:-1], layer_sizes[1:], layer_keys):
        network_params.append(random_layer_params(in_size, out_size, key))
    return network_params


def relu(x: jnp.DeviceArray) -> jnp.DeviceArray:
    """Rectified-Linear Unit."""
    return jnp.maximum(0, x)


@functools.partial(jax.vmap, in_axes=(None, 0))
def predict(
    params: List[Tuple[jnp.DeviceArray]],
    image: jnp.DeviceArray,
) -> jnp.DeviceArray:
    """Forward prediction with an MLP denoted by params."""
    x: jnp.DeviceArray = image
    for w, b in params[:-1]:
        x = relu(jnp.dot(w, x) + b)
    # last layer has no activation
    last_w, last_b = params[-1]
    x = relu(jnp.dot(last_w, x) + last_b)
    # log of the sum of exponentials of input elements
    return x - jax.scipy.special.logsumexp(x)


def one_hot(
    x: jnp.DeviceArray,
    k: int,
    dtype=jnp.float32,
) -> jnp.DeviceArray:
    """One-hot encoding of size k."""
    return jnp.array(x[:, None] == jnp.arange(k), dtype)


def accuracy(
    params: List[Tuple[jnp.DeviceArray]],
    images: jnp.DeviceArray,
    targets: jnp.DeviceArray,
) -> jnp.DeviceArray:
    """Accuracy of one-hot image prediction compared to target."""
    target_class = jnp.argmax(targets, axis=1)
    predict_class = jnp.argmax(predict(params, images), axis=1)
    return jnp.mean(predict_class == target_class)


def loss(
    params: List[Tuple[jnp.DeviceArray]],
    images: jnp.DeviceArray,
    targets: jnp.DeviceArray,
) -> jnp.DeviceArray:
    """Categorical cross entropy?"""
    return -jnp.mean(predict(params, images) * targets)


@jax.jit
def update(
    params: List[Tuple[jnp.DeviceArray]],
    images: jnp.DeviceArray,
    targets: jnp.DeviceArray,
    learning_rate: float = 1e-3,
) -> List[Tuple[jnp.DeviceArray]]:
    grads = jax.grad(loss)(params, images, targets)
    return [
        (
            # updated weight
            w - learning_rate * dw,
            # updated bias
            b - learning_rate * db,
        )
        for (w, b), (dw, db) in zip(params, grads)
    ]


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

DATA_DIR = "/tmp/tfds"

# Load the full MNIST dataset
mnist_data, info = tfds.load(
    name="mnist", batch_size=-1, data_dir=DATA_DIR, with_info=True
)
mnist_data = tfds.as_numpy(mnist_data)
train_data = mnist_data["train"]
test_data = mnist_data["test"]
num_labels = info.features["label"].num_classes
h, w, c = info.features["image"].shape
num_pixels = h * w * c

# Train dataset
train_labels = train_data["label"]
train_labels = one_hot(train_labels, num_labels)
train_images = train_data["image"]
train_images = jnp.reshape(train_images, (len(train_images), num_pixels))

# Test dataset
test_labels = test_data["label"]
test_labels = one_hot(test_labels, num_labels)
test_images = test_data["image"]
test_images = jnp.reshape(test_images, (len(test_images), num_pixels))

print(f"Train: {train_images.shape}, {train_labels.shape}")
print(f"Test: {test_images.shape}, {test_labels.shape}")
print(f"num_pixels {num_pixels}")
print(f"num_labels {num_labels}")


In [ ]:
def get_train_batches(batch_size):
    """Dataloader function returns batches of training data."""
    ds = tfds.load(name="mnist", split="train", as_supervised=True, data_dir=DATA_DIR)
    ds = ds.batch(batch_size).prefetch(1)
    return tfds.as_numpy(ds)

In [ ]:
import wandb

os.environ['WANDB_NOTEBOOK_NAME'] = 'minisweep'

wandb.login()

In [ ]:
WANDB_PROJECT = "minisweep"

# https://docs.wandb.ai/guides/sweeps/configuration
sweep_config: Dict = {
    "method": "random",
    # "method": 'grid',
    # "method": 'bayes',
    "metric": {
        "name": "test_accuracy",
        "goal": "maximize",
    },
    "parameters": {
        "learning_rate": {
            "distribution": "log_uniform",  # _values",
            "min": 1e-6,
            "max": 1e-3,
        },
        "layers": {
            "values": [
                "784,128,128,10",
                "784,512,512,10",
            ],
        },
        "num_epochs": {
            "distribution": "q_log_uniform",  # _values",
            "q": 1,
            "min": 5,
            "max": 30,
        },
        "batch_size": {
            "value": 128,
        },
        "num_pixels": {
            "value": 784,
        },
        "num_labels": {
            "value": 10,
        },
    },
}


In [ ]:
def train(config=None):
    """Main training loop."""
    with wandb.init(project=WANDB_PROJECT, config=config):

        params = init_network_params(config.layers, RANDOM_KEY)

        # training loop
        for epoch in range(config.num_epochs):
            start_time = time.time()
            for images, labels in get_train_batches(config.batch_size):
                images = jnp.reshape(images, (len(images), config.num_pixels))
                labels = one_hot(labels, config.num_labels)
                params = update(
                    params, images, labels, learning_rate=config.learning_rate
                )
            epoch_time = time.time()

            # Re-calculating on entire dataset, this is super inneficient
            train_accuracy = accuracy(params, train_images, train_labels)
            test_accuracy = accuracy(params, test_images, test_labels)
            _out = {
                "epoch": epoch,
                "start_time": start_time,
                "duration": epoch_time - start_time,
                "train_accuracy": train_accuracy,
                "test_accuracy": test_accuracy,
            }
            wandb.log(_out)
            pprint.pprint(_out)


In [ ]:
from dataclasses import dataclass


def test_train():
    """Test the train function locally."""

    @dataclass
    class TestConfig:
        layers: str = "784,10"
        learning_rate: float = 1e-3
        batch_size: int = 128
        num_epochs: int = 1
        num_pixels: int = 784
        num_labels: int = 10

    os.environ['WANDB_SILENT']="true"
    train(TestConfig())
    os.environ['WANDB_SILENT']="false"


test_train()


In [ ]:
sweep_id = wandb.sweep(sweep_config, project=WANDB_PROJECT)
wandb.agent(sweep_id, train, count=2)